<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/annelie/notebooks/Huggingface_QA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2: Evaluate Dataset

In [1]:
!pip install word2number

import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests
from word2number import w2n
import re
import pandas as pd

# Gemini API Setup
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# Read dataset file
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url)

data.head()

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=ff618407a9a5a55a10e541e12b154c26ea91b161d29a2e7073437f7de6a5d8ce
  Stored in directory: /root/.cache/pip/wheels/cd/ef/ae/073b491b14d25e2efafcffca9e16b2ee6d114ec5c643ba4f06
Successfully built word2number


,type,question,options,intended_answer,context,timestamp
0,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Absolutely, I'm totally okay with data process...",2024-12-26 13:54:58.593
1,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yeah, you have my full consent for data proces...",2024-12-26 13:54:58.593
2,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Oh, data processing consent? Yes, you can go r...",2024-12-26 13:54:58.593
3,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,Regarding your request for data processing con...,2024-12-26 13:54:58.593
4,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yes, please proceed with the data processing, ...",2024-12-26 13:54:58.593


In [2]:
def convert_numbers_in_text(text):
    # Regular expression to find number words contained in questionnaires
    pattern = r'(two thousand|two hundred one|two hundred|fifty-one|thirty-one|twenty-one|sixteen|fifteen|eleven|thirty|twenty|fifty|forty|sixty|ten|five|six|one)'
    # Interesting finding: Regex only works if longer words are in order before shorter that contain similar parts, e.g. fifty-one has to be in front of fifty to work as intended

    def convert(match):
        word = match.group(0)
        try:
            # Convert the word to number
            return str(w2n.word_to_num(word))
        except ValueError:
            return word

    # Replace all number words in the text with their integer equivalents
    converted_text = re.sub(pattern, convert, text, flags=re.IGNORECASE)

    # Now convert ranges like 'twenty to thirty' into '20-30'
    converted_text = re.sub(r'(\d+)\s*(to|and)\s*(\d+)', r'\1-\3', converted_text)

    # Replace text
    # Todo: Dafür noch ne bessere Lösung finden, das ist eig nur n Beispiel und geht auch bei ähnlichen Sätzen nicht
    converted_text = converted_text.replace('more than 2000', 'larger than 2000')
    converted_text = converted_text.replace('More than 2000', 'Larger than 2000')

    return converted_text


def is_exact_or_phrase_match(option, text):
    # Escape the option to handle special characters
    escaped_option = re.escape(option.strip())

    # Pattern to match the option as a full word or part of a phrase
    pattern = rf'\b(?:\w+\s+)*{escaped_option}(?:\s+\w+)*\b'

    # Search for the pattern in the text (case-insensitive)
    return re.search(pattern, text, re.IGNORECASE) is not None

In [3]:
from transformers import pipeline

qa_pipeline1 = pipeline("question-answering", model="deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
qa_pipeline2 = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
qa_pipeline3 = pipeline("question-answering", model='google-bert/bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
from pickle import NONE
!pip install dateparser
import dateparser
from datetime import datetime
from datetime import timedelta

def predict_answers(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0
    qa_pipeline = pipeline

    for _, row in df.iterrows():

        predictions = []

        # Regex-check only for single- and multi-select questions
        if row['options']:  # Evaluates to False if options is None or empty
            converted_context = convert_numbers_in_text(row['context'])

            for option in row['options']:
                # Check for exact match or part of a phrase
                exact_match = is_exact_or_phrase_match(option, converted_context)
                if exact_match:
                    predictions.append((option, 0.95))  # 95% confidence for exact match
                else:
                    result = qa_pipeline(question=row['question'], context=f"{converted_context} {option}")
                    predictions.append((option, result['score']))

        is_correct = False

        # Handle different question types
        if row['type'] == "SINGLE_SELECT":
            # Predict the option with the highest confidence
            predicted_option, confidence = max(predictions, key=lambda x: x[1])
            is_correct = predicted_option == row['intended_answer']

        elif row['type'] == "MULTI_SELECT":
            predicted_option = [option for option, score in predictions if score >= 0.95]
            is_correct = set(predicted_option) == set(row['intended_answer'])

        elif row['type'] == "DATE":
            try:
                # Basis-Timestamp aus der Dataframe-Spalte (Unix-Timestamp)
                base_timestamp = pd.Timestamp(row['timestamp'], unit='ms')

                # Extrahiere Zeitangabe aus dem Kontext
                extracted_time = qa_pipeline(question=row['question'], context=row['context'])['answer']

                # Konvertiere extrahierte Zeitangabe in Sekunden
                parsed_seconds = dateparser.parse(
                    extracted_time,
                    settings={'RELATIVE_BASE': base_timestamp.to_pydatetime()}
                )
                if not parsed_seconds:
                    raise ValueError(f"Unable to parse date from extracted time: {extracted_time}")

                predicted_option = parsed_seconds
                intended_seconds = int(row['intended_answer'])
                intended_date = base_timestamp + timedelta(seconds=intended_seconds)

                # Vergleich der vorhergesagten und intendierten Daten
                is_correct = predicted_option.date() == intended_date.date()

            except Exception as e:
                print(f"[ERROR] DATE question processing failed: {e}")

        elif row['type'] == "NUMBER":
            try:
                predicted_option = qa_pipeline(question=row['question'], context=row['context'])['answer']
                is_correct = predicted_option == row['intended_answer']
            except Exception as e:
                print(f"[ERROR] NUMBER question failed: {e}")

        elif row['type'] == "TEXT":
            continue
            # predicted_option = row['context']
            # is_correct = True

        # Output incorrect predictions
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

    # Calculate accuracy
    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.0 MB/s eta 0:00:00


## Evaluate Dataset with Pre-trained Models

In [ ]:
accuracy = predict_answers(data, qa_pipeline1)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.
Context: Okay, so when you're talking about the customer group, we're definitely thinking about folks who are either a wholesaler or a distributor, that’s who we're focused on here.
Correct: Wholesaler, Distributor, Predicted: End User

Context: Well, the customer group we need to consider in this case, is made up of both the wholesaler and the distributor types, you see.
Correct: Wholesaler, Distributor, Predicted: End User

Context: If we're looking at the different customer groups, the ones we're targeting are the wholesaler and the distributor, plain and simple.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Context: So, the customer group that we’ve got is comprised of, let me see, Consultants, naturally, then the Planners, and you can't forget the Architects, they're a big part of it.
Correct: Consultant, Planner, Architect, Predicted: Wholesaler, Distributor

Context: The various customer groups we

In [7]:
accuracy = predict_answers(data, qa_pipeline2)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Context: Well, the customer group we need to consider in this case, is made up of both the wholesaler and the distributor types, you see.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Context: If we're looking at the different customer groups, the ones we're targeting are the wholesaler and the distributor, plain and simple.
Correct: Wholesaler, Distributor, Predicted: End User

Context: For this particular situation, the customer group includes, specifically, those who act as a wholesaler and those who are distributors, yeah that's it.
Correct: Wholesaler, Distributor, Predicted: End User

Context: To be clear, the intended customer group comprises both the wholesaler category and the distributor category, those are the ones we're thinking about.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Context: Well, when we talk about the customer groups, we're mainly looking at Consultants, Planners, and Architects, those are the fo

In [9]:
accuracy = predict_answers(data, qa_pipeline3)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.
Context: Absolutely not, I do not give my consent for data processing
Correct: No, Predicted: Yes

Context: Okay, so when you're talking about the customer group, we're definitely thinking about folks who are either a wholesaler or a distributor, that’s who we're focused on here.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Context: To be clear, the intended customer group comprises both the wholesaler category and the distributor category, those are the ones we're thinking about.
Correct: Wholesaler, Distributor, Predicted: R&D

Context: Regarding who we're thinking of as our customers, we can definitely say that Consultants are one group, then there are also Planners, and of course the Architects, those three are key.
Correct: Consultant, Planner, Architect, Predicted: End User

Context: "Well, let me tell you, for follow-up, we've got a few options on the table: we could send out an Email, or maybe w

## Fine-tune SQUAD-Model

In [42]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
!pip install datasets
from datasets import Dataset

def prepare_squad_data(df):
    squad_data = {"data": []}

    for idx, row in df.iterrows():
        question = row["question"]
        context = row["context"]
        intended_answer = row["intended_answer"]
        if (row['type'] == 'DATE') or (row['type'] == 'TEXT'):
          continue

        start_positions = []
        end_positions = []

        if isinstance(intended_answer, list):
            for answer in intended_answer:
                answer = str(answer)  # Konvertiere jeden Eintrag in einen String
                start = context.find(answer)
                if start != -1:
                    start_positions.append(start)
                    end_positions.append(start + len(answer))
                else:
                    print(f"Warnung: Antwort '{answer}' nicht im Kontext enthalten. Überspringe Datensatz.")
                    continue
        # Single-Select (nur ein String)
        elif isinstance(intended_answer, str):
            answer = intended_answer
            start = context.find(answer)
            if start != -1:
                start_positions.append(start)
                end_positions.append(start + len(answer))
            else:
                print(f"Warnung: Antwort '{answer}' nicht im Kontext enthalten. Überspringe Datensatz.")
                continue
        else:
            # Wenn der Typ von intended_answer weder Liste noch String ist
            print(f"Warnung: 'intended_answer' hat ein ungültiges Format in Zeile {idx}. Überspringe Datensatz.")
            continue

        squad_data["data"].append({
    "paragraphs": [
        {
            "context": context,
            "qas": [
                {
                    "question": question,
                    "id": f"q_{idx}",
                    "answers": [
                        {"text": answer, "answer_start": start} for answer in intended_answer
                    ],
                    "is_impossible": False
                        }
                    ]
                }
            ]
        })

    return squad_data

def tokenize_squad_data(squad_data, tokenizer, max_length=512):
    """
    Tokenisiert das vorbereitete SQuAD-Dataset.
    """
    # Telefonnummern-Regex (für verschiedene internationale Formate)
    phone_regex = re.compile(r"\+?\d{1,3}[-\s]?\(?\d{1,4}\)?[-\s]?\d{1,4}[-\s]?\d{1,9}")

    tokenized_examples = []

    for data in squad_data["data"]:
        for paragraph in data["paragraphs"]:
            # Kontext vorbereiten: Telefonnummern markieren
            context = paragraph["context"]
            context_matches = phone_regex.findall(context)
            for match in context_matches:
                context = context.replace(match, f"[PHONE_TOKEN_{match}]")

            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa["answers"]

                # Antworten vorbereiten: Telefonnummern markieren
                processed_answers = []
                for answer in answers:
                    answer_text = answer["text"]
                    if phone_regex.match(answer_text):
                        answer_text = f"[PHONE_TOKEN_{answer_text}]"
                    processed_answers.append({
                        "text": answer_text,
                        "answer_start": context.find(answer_text)  # Aktualisierte Startposition
                    })

                # Extrahiere die Positionen der Antworten
                start_positions = [answer["answer_start"] for answer in processed_answers]
                answer_texts = [answer["text"] for answer in processed_answers]

                # Tokenize Frage und Kontext
                tokenized_example = tokenizer(
                    question,
                    context,
                    max_length=max_length,
                    truncation="only_second",  # Kontext wird bei Überlänge abgeschnitten
                    padding="max_length",
                    return_offsets_mapping=True
                )

                # # Berechne Token-Startpositionen der Antworten
                # offsets = tokenized_example.pop("offset_mapping")
                # token_start_positions = []
                # token_end_positions = []
                # for start_position, answer_text in zip(start_positions, answer_texts):
                #     for i, (offset_start, offset_end) in enumerate(offsets):
                #         if offset_start <= start_position < offset_end:
                #             token_start_positions.append(i)
                #             token_end_positions.append(i + len(answer_text))
                #             break

                # # Tokenisierte Daten speichern
                # if token_start_positions:
                #     tokenized_example["start_positions"] = token_start_positions[0]
                #     tokenized_example["end_positions"] = token_end_positions[0]
                #     tokenized_examples.append(tokenized_example)
                # else:
                #     # Warnung bei nicht gefundenen Antworten
                #     print(f"Warning: Answer not found in context for question: {question}")
                #     print(f"Intended answer: {answer_texts}")
                #     print(f"Context: {context}")

                # Berechne Token-Startpositionen der Antworten (case-insensitive)
                token_start_positions = []
                token_end_positions = []
                context_lower = context.lower()  # Kontext in Kleinbuchstaben
                answer_texts_lower = [answer_text.lower() for answer_text in answer_texts]  # Antworten in Kleinbuchstaben

                for answer_text, answer_text_lower in zip(answer_texts, answer_texts_lower):
                    start_position = context_lower.find(answer_text_lower)  # Suche case-insensitive
                    if start_position != -1:
                        # Füge die tatsächliche Startposition und Endposition basierend auf dem Originaltext hinzu
                        token_start_positions.append(start_position)
                        token_end_positions.append(start_position + len(answer_text))
                    else:
                        # Antwort wurde nicht gefunden
                        print(f"Warning: Answer not found in context for question: {question}")
                        print(f"Intended answer: {answer_text}")
                        print(f"Context: {context}")

                # Tokenisierte Daten speichern
                tokenized_example["start_positions"] = token_start_positions[0] if token_start_positions else 0  # Handle empty list
                tokenized_example["end_positions"] = token_end_positions[0] if token_end_positions else 0  # Handle empty list
                tokenized_examples.append(tokenized_example)


    return tokenized_examples

In [43]:
# Aufteilen in Trainings- und Testdaten, Testdaten werden nicht umgewandelt
train_data, test_data = train_test_split(data, test_size=0.3, stratify=data["type"], random_state=42)

# Trainings- und Validierungsdaten vorbereiten
squad_data = prepare_squad_data(train_data)

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
tokenized_data = tokenize_squad_data(squad_data, tokenizer)

train_data, val_data = train_test_split(tokenized_data, test_size=0.3, random_state=42)

# Dataset Klasse für Huggingface
class SquadDataset:
    def __init__(self, data):
        self.data = Dataset.from_dict({k: [v] for k, v in data[0].items()})
        for item in data[1:]:
            self.data = self.data.add_item(item)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": item["input_ids"],
            "attention_mask": item["attention_mask"],
            "start_positions": item["start_positions"],
            "end_positions": item["end_positions"]
        }

# Huggingface Datasets erstellen
train_dataset = SquadDataset(train_data)
val_dataset = SquadDataset(val_data)
print(len(train_dataset))

Warnung: Antwort 'Very unsatisfied' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Wholesaler, Distributor' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Consultant, Planner, Architect' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Consultant, Planner, Architect' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To check project progress' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To communicate with team members' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To create new tasks' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To update task status' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To communicate with team members' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'To create new tasks' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Satisfied' nicht im Konte

In [45]:
# Modell laden
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# TrainingArguments definieren
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

# Trainer erstellen
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-45-074ec5a41dc2>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [46]:
# Training starten
trainer.train()

# Save model
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./results /content/drive/MyDrive

Epoch,Training Loss,Validation Loss
1,6.032800,5.860127
2,5.880600,5.784520
3,5.671600,5.734552


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
# Model laden und Pipeline erstellen
import os

results_path = '/content/drive/MyDrive/results'
folders = [os.path.join(results_path, folder) for folder in os.listdir(results_path) if os.path.isdir(os.path.join(results_path, folder))]
latest_folder = max(folders, key=os.path.getctime)

model_path = latest_folder
print(f"Der zuletzt erstellte Ordner ist: {model_path}")

model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

qa_pipeline_squad = pipeline("question-answering", model=model, tokenizer=tokenizer)

Der zuletzt erstellte Ordner ist: /content/drive/MyDrive/results/checkpoint-171


Device set to use cuda:0


In [48]:
accuracy = predict_answers(test_data, qa_pipeline_squad)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.
Context: Honestly, I’m mostly just hanging out on this app to go over some product information; that's really the core of what I’m doing today – gotta get the specs and all that!
Correct: To review product information, Predicted: Other

Context: Of course, making sure we can stay in touch is important! The number you're looking for is +33-814-501-1126, and I’m available most of the time.
Correct: +33-814-501-1126, Predicted: I’

Context: Alright, so to answer your question directly, you can use the phone number +49-778-336-7278 if you need to get in touch with me, that's the best way.
Correct: +49-778-336-7278, Predicted: that's the best way.

Context: So, concerning the average size of the trade fair team, we're talking a number that goes beyond 40, yeah, that's about right.
Correct: more than 40, Predicted: 11-15

Context: "Absolutely, I'm happy to give you my phone number, it's +1-395-371-5592. Please feel free to call if you need 

## Evaluate Dataset with a Generative Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from fuzzywuzzy import fuzz
import pandas as pd
import dateparser
from datetime import timedelta

# Initialisiere T5-Modell und Tokenizer
def initialize_t5_pipeline(model_name="t5-small"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    def t5_pipeline(input_text):
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(inputs.input_ids, max_length=50)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return t5_pipeline

# Lade die T5-Pipeline
t5_pipeline = initialize_t5_pipeline()

def similarity(option, answer):
    """Berechnet die Ähnlichkeit zwischen zwei Texten."""
    return fuzz.ratio(option.lower(), answer.lower())

def predict_answers(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0

    for _, row in df.iterrows():

        predictions = []
        converted_context = row['context']

        # Frage-Kontext-Format für T5 erstellen
        t5_input = f"question: {row['question']} context: {converted_context}"

        # Antworten vorhersagen, abhängig vom Fragetyp
        if row['type'] == "SINGLE_SELECT":
            for option in row['options']:
                # Generiere Antwort und berechne Ähnlichkeit
                generated_answer = pipeline(f"question: {row['question']} context: {converted_context} {option}")
                predictions.append((option, similarity(option, generated_answer)))

            # Wähle die Option mit der höchsten Ähnlichkeit
            predicted_option, _ = max(predictions, key=lambda x: x[1])
            is_correct = predicted_option == row['intended_answer']

        elif row['type'] == "MULTI_SELECT":
            generated_answer = pipeline(t5_input)
            # Zerlege generierte Antwort in mögliche Optionen (z. B. durch Kommas getrennt)
            generated_answers = [ans.strip() for ans in generated_answer.split(",")]
            predicted_option = [option for option in row['options'] if option in generated_answers]
            is_correct = set(predicted_option) == set(row['intended_answer'])

        elif row['type'] == "DATE":
            try:
                base_timestamp = pd.Timestamp(row['timestamp'], unit='ms')
                extracted_time = pipeline(t5_input)

                # Extrahiere und parse Datum
                parsed_date = dateparser.parse(
                    extracted_time,
                    settings={'RELATIVE_BASE': base_timestamp.to_pydatetime()}
                )
                if not parsed_date:
                    raise ValueError(f"Unable to parse date from extracted time: {extracted_time}")

                predicted_option = parsed_date.date()
                intended_date = (base_timestamp + timedelta(seconds=int(row['intended_answer']))).date()
                is_correct = predicted_option == intended_date

            except Exception as e:
                print(f"[ERROR] DATE question processing failed: {e}")
                is_correct = False

        elif row['type'] == "NUMBER":
            try:
                generated_answer = pipeline(t5_input)
                predicted_option = float(generated_answer) if generated_answer.isdigit() else None
                is_correct = predicted_option == float(row['intended_answer'])
            except Exception as e:
                print(f"[ERROR] NUMBER question failed: {e}")
                is_correct = False

        elif row['type'] == "TEXT":
            generated_answer = pipeline(t5_input)
            is_correct = similarity(generated_answer, row['intended_answer']) > 90  # Threshold für Ähnlichkeit

        # Ausgabe falscher Vorhersagen
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

    # Berechne Genauigkeit
    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"[INFO] Accuracy: {accuracy:.2%}")
    return accuracy

# Beispiel DataFrame erstellen
data = {
    "question": ["Was ist die Hauptstadt von Deutschland?"],
    "context": ["Berlin ist die Hauptstadt von Deutschland."],
    "type": ["SINGLE_SELECT"],
    "options": [["Berlin", "München", "Hamburg"]],
    "intended_answer": ["Berlin"],
    "timestamp": [None]  # Nur für DATE-Fragen relevant
}
df = pd.DataFrame(data)

# Vorhersagen durchführen
accuracy = predict_answers(df, t5_pipeline)


## Evaluate continuous text

## Interesting Findings

*   Prediction of names very bad, because no deeper meaning --> fixed by checking for exact matches
  * Maybe implement name interpreter later?
*   Numerical values (size of company) prediction very bad

* QA Pipelines
  * Pipeline 2 und 3 haben nur eine accuracy von ungefähr 60 %

